In [7]:
# Importar as bibliotecas 
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [8]:
# Verificar os arquivos contidos na pasta
arquivos = os.listdir()
print(arquivos)

['ocorrencia.csv', 'Projeto.ipynb', 'Tabela_localizacao.csv']


In [9]:
ocorrencia = pd.read_csv('ocorrencia.csv',encoding='latin-1', sep=';')
display(ocorrencia)

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,83097,83097,83097,83097,83097,ACIDENTE,-2.352.777.777.777,-4.563.944.444.444,PARAIBUNA,SP,...,31/12/2023,18:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,83085,83085,83085,83085,83085,INCIDENTE,-9.868.333.333.333,-6.789.805.555.555,RIO BRANCO,AC,...,13/12/2023,11:33:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2,83074,83074,83074,83074,83074,INCIDENTE,-1.962.444.444.444,-4.397.194.444.444,BELO HORIZONTE,MG,...,20/12/2023,20:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
3,83063,83063,83063,83063,83063,INCIDENTE,-1.587.111.111.111,-4.791.861.111.111,BRASÍLIA,DF,...,29/12/2023,21:20:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
4,83051,83051,83051,83051,83051,INCIDENTE,-4.871.388.888.888,-668.975,CARAUARI,AM,...,09/12/2023,18:30:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9816,28377,28377,28377,28377,28377,ACIDENTE,***,***,IMPERATRIZ,MA,...,27/01/2007,11:18:00,NaN,FINALIZADA,A-053/CENIPA/2009,SIM,16/11/2009,2,1,NÃO
9817,28375,28375,28375,28375,28375,ACIDENTE,-236.122.222.222,-51.385,APUCARANA,PR,...,26/01/2007,18:15:00,NaN,FINALIZADA,A-021/CENIPA/2010,SIM,15/08/2007,1,1,NÃO
9818,28355,28355,28355,28355,28355,ACIDENTE,-172.541.666.667,-438.536.111.111,BOCAIÚVA,MG,...,18/01/2007,09:10:00,NaN,FINALIZADA,A-001/CENIPA/2010,SIM,19/03/2010,11,1,NÃO
9819,28335,28335,28335,28335,28335,ACIDENTE,-233.444.444.444,-510.633.333.333,LONDRINA,PR,...,19/01/2007,12:30:00,NaN,FINALIZADA,A-169/CENIPA/2013,SIM,16/12/2013,1,1,NÃO


In [10]:
# Selecionar apenas as colunas importantes para a raspagem de dados
localizacao_geo = ocorrencia[['ocorrencia_cidade', 'ocorrencia_uf']]
# Limitando o conjunto de dados apenas para 10 linhas para teste
localizacao_geo = localizacao_geo[:10]

In [11]:
# Processo de raspagem de dados na internet

navegador = webdriver.Chrome()
navegador.maximize_window() 

localizacao_geo['localizacao'] = localizacao_geo['ocorrencia_cidade'] + ' - ' + localizacao_geo['ocorrencia_uf']

# inserir o loop de Latitude e Longitude
navegador.get("https://www.coordenadas-gps.net/")
for valores in localizacao_geo:
        if '-' not in valores: 
            tabela_geo = pd.DataFrame()
            localizacao_geo_filtro = localizacao_geo.loc[localizacao_geo['ocorrencia_cidade'] != 'NÃO IDENTIFICADA']
            localizacao_geo_filtro = localizacao_geo_filtro.loc[localizacao_geo['ocorrencia_cidade'] != 'ÁGUAS INTERNACIONAIS']
            tabela_geo['cidade'] = (localizacao_geo_filtro['ocorrencia_cidade'] + ' - ' + localizacao_geo['ocorrencia_uf']).unique()
            latitude = []
            longitude = []
            for localizar in tabela_geo['cidade']:
                procurar = navegador.find_element(By.ID, 'searchterm').send_keys(localizar)
                time.sleep(1)
                enter = navegador.find_element(By.CLASS_NAME, 'btn').click()
                time.sleep(1)
                coordenadas = navegador.find_element(By.CLASS_NAME, 'coordinatespin').text
                novo = coordenadas.strip('(').strip(')').strip('').split(',')         
                latitude += ([novo[0]])
                longitude += ([novo[1]])
                limpar = navegador.find_element(By.ID, 'searchterm').clear()               
            tabela_geo['latitude'] = latitude
            tabela_geo['longitude'] = longitude
            # Exportando a tabela com os resultados 
            tabela_geo.to_csv('Tabela_localizacao.csv', index=False, encoding = 'latin-1')
navegador.quit()



In [12]:
# Lendo e apresentando a tabela gerada
tabela = pd.read_csv('Tabela_localizacao.csv',encoding='latin-1', sep=',')
display(tabela)

,cidade,latitude,longitude
0,PARAIBUNA - SP,-23.387164,-45.662809
1,RIO BRANCO - AC,-23.387164,-45.662809
2,BELO HORIZONTE - MG,-19.919052,-43.938668
3,BRASÍLIA - DF,-15.797515,-47.891887
4,CARAUARI - AM,-4.877320,-66.896917
5,CAMPINAS - SP,-22.905082,-47.061333
6,CABO FRIO - RJ,-22.886751,-42.026254
7,FERNANDO DE NORONHA - PE,-3.844798,-32.426892
8,PRESIDENTE PRUDENTE - SP,-22.120784,-51.387320
9,PALMAS - TO,-10.249091,-48.324286
